In [7]:
# Import the custom library
import logic_algorithms as la
import sympy as sp
from sympy.logic.boolalg import Implies, And, Or, Not, Equivalent

# Plausability table

In [8]:
# Define the table with the plausibility orders
plausibility_table = {
    'p, q': {'states': ['x'], 'order': 2},
    'p, n_q': {'states': ['y'], 'order': 1},
    'n_p, q': {'states': ['z'], 'order': 2},
    'n_p, n_q': {'states': ['w'], 'order': 3}
}

styled_df = la.display_plausibility_table(plausibility_table)
styled_df


,"p, q","p, q̅","p̅, q","p̅, q̅"
0,,,,w
1,x,,z,
2,,y,,


## Contraction

Union of the prior most plausible word entailing the proposition and the prior most plausible word not entiling the proposition.

In [9]:
# Results of the contraction
print("Contraction with p: " + str(la.contraction(plausibility_table, 'p')))
print("Contraction with q: " + str(la.contraction(plausibility_table, 'q')))
print("Contraction with n_p: " + str(la.contraction(plausibility_table, 'n_p')))
print("Contraction with n_q: " + str(la.contraction(plausibility_table, 'n_q')))

Contraction with p: (['y'], ['z'])
Contraction with q: (['x'], ['y'])
Contraction with n_p: (['z'], ['y'])
Contraction with n_q: (['y'], ['x'])


## Revision

Most plausible state matching the revision

In [10]:
# Results of the revision
print("Revision with p: " + str(la.revision(plausibility_table, 'p')))
print("Revision with q: " + str(la.revision(plausibility_table, 'q')))
print("Revision with n_p: " + str(la.revision(plausibility_table, 'n_p')))
print("Revision with n_q: " + str(la.revision(plausibility_table, 'n_q')))

Revision with p: ['y']
Revision with q: ['x']
Revision with n_p: ['z']
Revision with n_q: ['y']


# Belief Base

# Truth table

In [11]:
# Define the symbols
r, p, l, q = sp.symbols('r p l q')

# Define the formulas in the knowledge base (KB)
KB = [
    Not(p),
    Implies(p, q),
    Implies(q, p),
    And(p, q),
    Or(p, q),
    Equivalent(p, q)
]

# Compute and display the truth table for the given knowledge base
la.compute_truth_table([p, q], KB)

,p,q,~p,"Implies(p, q)","Implies(q, p)",p & q,p | q,"Equivalent(p, q)"
0,F,F,T,T,T,F,F,T
1,F,T,T,T,F,F,T,F
2,T,F,F,F,T,F,T,F
3,T,T,F,T,T,T,T,T


## Revision of a Belief Base

In [12]:
# Define the symbols
p, q, r = sp.symbols('p q r')

# Define a belief base
belief_base = {p, q, Implies(p, q)}

# New belief
new_belief = Not(q)

# Check if adding the new belief keeps the belief base consistent
belief_base_with_new = belief_base | {new_belief}

if la.check_consistency(belief_base_with_new):
    print("The belief base with the new belief is CONSISTENT.")
    belief_base = belief_base_with_new
else:
    print("The belief base with the new belief is INCONSISTENT. Applying contraction...")
    # Apply contraction to make belief_base consistent with new_belief
    contracted_bases = la.contract_belief_base(belief_base, new_belief)
    # Each subset is a possible consistent belief base with the new belief
    for i, contracted_base in enumerate(contracted_bases):
        belief_base = set(contracted_base)
        belief_base.add(new_belief)
        # Display each consistent belief base
        print(f"Belief set {i + 1}:")
        for belief in belief_base:
            print(belief)
        print("-" * 30)

The belief base with the new belief is INCONSISTENT. Applying contraction...
Belief set 1:
~q
q
p
Implies(p, q)
------------------------------


## Revision of formulas

In [13]:
# Define the symbols
p, q = sp.symbols('p q')

# Define the formulas
formulas = {
    'p -> q': sp.Implies(p, q),
    'q -> p': sp.Implies(q, p),
    'p ∧ q': sp.And(p, q),
    'p ∨ q': sp.Or(p, q),
    'p ↔ q': sp.Equivalent(p, q)
}

# Given belief set revision with ¬p
belief_revision = sp.Not(p)

# Check each formula against the revised belief set
results = {}
for formula, expr in formulas.items():
    # Substitute ¬p into the expression
    revised_expr = expr.subs(p, False)
    # Evaluate the revised expression
    result = sp.simplify(revised_expr)
    results[formula] = result

# Print the results
print("Results of the belief revision with ¬p (simplification):")
for formula, result in results.items():
    print(f"{formula}: {result}")

# Compute and display the belief revision table
styled_table = la.compute_belief_revision_table(formulas, belief_revision)
styled_table


Results of the belief revision with ¬p (simplification):
p -> q: True
q -> p: ~q
p ∧ q: False
p ∨ q: q
p ↔ q: ~q


,p,q,Belief Revision: ¬p,p -> q,q -> p,p ∧ q,p ∨ q,p ↔ q,After ¬p: p -> q,After ¬p: q -> p,After ¬p: p ∧ q,After ¬p: p ∨ q,After ¬p: p ↔ q
0,F,F,T,T,T,F,F,T,T,T,F,F,T
1,F,T,T,T,F,F,T,F,T,F,F,T,F
2,T,F,F,F,T,F,T,F,T,T,F,F,T
3,T,T,F,T,T,T,T,T,T,F,F,T,F


## Contraction

A contraction p (inverted T) is the set of maximal subsets of A that do not imply p.

In [14]:
# Define the symbols
p, q = sp.symbols('p q')

# Define the belief base and the proposition
belief_base = {p, q, sp.Implies(p, q), sp.Implies(sp.Not(p), q)}
proposition = q

# Perform the contraction
contracted_bases = la.bb_contraction(belief_base, proposition)

# Each subset is a possible consistent belief base with the contraction
for i, contracted_base in enumerate(contracted_bases):
    belief_base = set(contracted_base)
    # Display each consistent belief base
    print(f"Belief set {i + 1}:")
    for belief in belief_base:
        print(belief)
    print("-" * 30)

Belief set 1:
Implies(p, q)
------------------------------
Belief set 2:
p
Implies(~p, q)
------------------------------


# Convert to CNF

In [15]:
# Define the symbols
a, b, c, d, e, f, r, p, s = sp.symbols('a b c d e f r p s')

# Define a list of logical expressions
expressions = [
    Equivalent(a, Or(c, e)),
    Implies(e, d),
    Implies(And(b, f), Not(c)),
    Implies(e, c),
    Implies(c, f),
    Implies(c, b),
    Implies(r, Or(p, s)),
]


# Convert the expressions to CNF
cnf_expressions = la.convert_to_cnf(expressions)

# Display the original and CNF expressions
for i, expr in enumerate(expressions):
    print(f"Original Expression {i+1}: {expr}")
    print(f"CNF {i+1}: {cnf_expressions[i]}")

Original Expression 1: Equivalent(a, c | e)
CNF 1: (a | ~c) & (a | ~e) & (c | e | ~a)
Original Expression 2: Implies(e, d)
CNF 2: d | ~e
Original Expression 3: Implies(b & f, ~c)
CNF 3: ~b | ~c | ~f
Original Expression 4: Implies(e, c)
CNF 4: c | ~e
Original Expression 5: Implies(c, f)
CNF 5: f | ~c
Original Expression 6: Implies(c, b)
CNF 6: b | ~c
Original Expression 7: Implies(r, p | s)
CNF 7: p | s | ~r
